In [35]:
import pandas as pd
import numpy as np

In [51]:
cd ../ml-latest

/Users/ioanwilliams/notebooks/Recommendation Systems/ml-latest


In [52]:
movies = pd.read_csv(r"movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [53]:
all_ratings = pd.read_csv(r"all_ratings.csv")
all_ratings.tail()

,Unnamed: 0,userId,movieId,rating
26025009,720,270897,171011,5.0
26025010,721,270897,171495,3.0
26025011,722,270897,172217,1.0
26025012,723,270897,174055,4.0
26025013,724,270897,174685,4.0


In [54]:
all_ratings.count()

Unnamed: 0    26025014
userId        26025014
movieId       26025014
rating        26025014
dtype: int64

In [ ]:
R_df = all_ratings.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
R_df.tail()

In [41]:
ratings_matrix = R_df.as_matrix()
#user_ratings_mean = np.mean(ratings_matrix, axis = 1)
#R_demeaned = ratings_matrix - user_ratings_mean.reshape(-1, 1)

/Users/ioanwilliams/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [42]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(ratings_matrix, k = 50)
sigma = np.diag(sigma)

In [43]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) #+ user_ratings_mean.reshape(-1, 1)

In [44]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False) # UserID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.userId == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )

    print('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies_df[~movies_df['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [45]:
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [49]:
already_rated, predictions = recommend_movies(preds_df, 672, movies, all_ratings, 15)

User 672 has already rated 672 movies.
Recommending highest 15 predicted ratings movies not already rated.


In [50]:
predictions

,movieId,title,genres
1131,1625,"Game, The (1997)",Drama|Mystery|Thriller
2890,4022,Cast Away (2000),Drama
2432,3408,Erin Brockovich (2000),Drama
3022,4226,Memento (2000),Mystery|Thriller
2796,3897,Almost Famous (2000),Drama
972,1370,Die Hard 2 (1990),Action|Adventure|Thriller
6667,68157,Inglourious Basterds (2009),Action|Drama|War
1378,2000,Lethal Weapon (1987),Action|Comedy|Crime|Drama
4570,7143,"Last Samurai, The (2003)",Action|Adventure|Drama|War
4974,8529,"Terminal, The (2004)",Comedy|Drama|Romance


In [48]:
already_rated.head()

,Unnamed: 0,userId,movieId,rating,title,genres
189,189,672,2028,5.0,Saving Private Ryan (1998),Action|Drama|War
633,633,672,109374,5.0,"Grand Budapest Hotel, The (2014)",Comedy|Drama
148,148,672,1466,5.0,Donnie Brasco (1997),Crime|Drama
126,126,672,1265,5.0,Groundhog Day (1993),Comedy|Fantasy|Romance
184,184,672,2005,5.0,"Goonies, The (1985)",Action|Adventure|Children|Comedy|Fantasy
